In [ ]:
# 필요한 라이브러리 import
import os
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl

# 이미지 파일이 있는 경로
image_folder = r'C:\Users\USER\Desktop\강서구load\35816(김해)'

# KeplerGl 맵 초기화
map_1 = KeplerGl(height=500)  # 지도 크기를 설정할 수 있습니다.

# 이미지 파일을 불러와 지도에 추가
for root, dirs, files in os.walk(image_folder):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png'):
            image_path = os.path.join(root, file)
            map_1.add_data(data=pd.DataFrame({'Image': [image_path]}), name=file)

# KeplerGl 맵 표시
map_1


In [ ]:
# Load an empty map
from keplergl import KeplerGl
map_1 = KeplerGl()
map_1

In [ ]:
# KeplerGl 맵 초기화
map_1 = KeplerGl(height=800, width=1000, config=map_config)

# 이미지와 XML 데이터를 KeplerGl에 추가
map_1.add_data(data=image_data, name="Images")
map_1.add_data(data=xml_data, name="XML Files")

# KeplerGl 시각화를 웹 브라우저에서 열고 시각화
map_1
map_1.save_to_html(file_name="Boundarys.html")

In [ ]:
#부산구 추가
busan_gu='c:/Users/OFFICE/Downloads/busan_gu.json'
with open(busan_gu,'r',encoding='utf-8') as busan_gu_file:
    geobusan=json.load(busan_gu_file)

map_1.add_data(data=geobusan,name="Busan_Boundarys")
map_1.save_to_html(file_name="Boundarys.html")

In [ ]:
import h3
import geopandas as gpd
import rasterio as rio
import pyproj


# 부산시 경계의 위경도 좌표
#WGS84 -> TM 좌표로  DEM데이터인덱스 접근
#값 리턴
dem_path='C:/Users/OFFICE/Desktop/한반도90m_GRS80.img'
dem_data=rio.open(dem_path)
wgs84=pyproj.CRS("EPSG:4326")
tm=pyproj.CRS(dem_data.crs)
transformer=pyproj.Transformer.from_crs(wgs84,tm,always_xy=True)
#좌표변환 wgs84 입력 -> TM 좌표 리턴
def Get_WGS84(longitude,latitude):
    return transformer.transform(longitude,latitude)

import json

busan_simplify_path='C:/Users/OFFICE/Desktop/navigation/부산광역시/부산시경계/busan_gu.json'
#busan_simplify=[]


res=10




with open(busan_simplify_path, 'r',encoding='UTF-8') as file:
    geodata= json.load(file)
h3_hexagon=[]
for geodata in geodata['features']:
    if geodata['geometry']['type']=='MultiPolygon':
        tmp=list(geodata['geometry']['coordinates'])
        for t in tmp:
            t={
                'type':'Polygon','coordinates':t
            }
            h3_hexagon=h3_hexagon+list(h3.polyfill_geojson(t,res))
    else: #Polygon일때
        h3_hexagon=h3_hexagon+list(h3.polyfill_geojson(geodata['geometry'],res))



height=[]
AABB=dem_data.bounds
data=dem_data.read(1)
busan_h3_cells=h3_hexagon
for index in busan_h3_cells:
    try:
        coordinates=h3.h3_to_geo(index)
        coordinate=[coordinates[1],coordinates[0]]
        coordinates=coordinate
        coordinates=Get_WGS84(coordinates[0],coordinates[1])
        index=dem_data.index(coordinates[0],coordinates[1])
        height.append(data[index[0]][index[1]])
    except:
        height.append(0)

d=pd.Series(busan_h3_cells,height)
d.name="hex_id"
d={
    "hex_id":busan_h3_cells,"height":height,
}




busan_gdf=pd.DataFrame(d)

map_1.add_data(data=busan_gdf,name="H3 cells")
map_1.save_to_html(file_name="h3_.html")

In [ ]:
# import h3
# import geopandas as gpd
# import rasterio as rio
# import pyproj


# # 부산시 경계의 위경도 좌표
# #WGS84 -> TM 좌표로  DEM데이터인덱스 접근
# #값 리턴
# dem_path='C:/Users/OFFICE/Desktop/한반도90m_GRS80.img'
# dem_data=rio.open(dem_path)
# wgs84=pyproj.CRS("EPSG:4326")
# tm=pyproj.CRS(dem_data.crs)
# transformer=pyproj.Transformer.from_crs(wgs84,tm,always_xy=True)
# #좌표변환 wgs84 입력 -> TM 좌표 리턴
# def Get_WGS84(longitude,latitude):
#     return transformer.transform(longitude,latitude)

# ########################################################################################################

# busan_boundary_coords = {
#     "type": "Polygon",
#     "coordinates": [
#         [
#             (35.035662, 128.894316),
#             (35.310800, 128.894316),
#             (35.310800, 129.309196),
#             (35.035662, 129.309196),
#             (35.035662, 128.894316)
#         ]
#     ]
# }

# # 부산시 경계를 나타내는 GeoDataFrame 생성


# # 부산시 경계 내에 있는 "uber h3" 그리드 생성
# import json

# busan_simplify_path='C:/Users/OFFICE/Desktop/navigation/부산광역시/부산시경계/busan_main_simplify.json'
# #busan_simplify=[]
# with open(busan_simplify_path, 'r',encoding='UTF-8') as file:
#     geodata= json.load(file)


# polygon=[(point[1],point[0]) for point in geodata['features'][0]['geometry']['coordinates'][0]] 
# polygon2=([(point[1],point[0]) for point in geodata['features'][1]['geometry']['coordinates'][0]])
# polygon_geojson={
#     "type": "Polygon",
#     "coordinates":[polygon]
# }   
# polygon_geojson2={
#     "type": "Polygon",
#     "coordinates":[polygon2]
# }   
# h3_resolution = 8  # 그리드 해상도 설정
# busan_h3_cells = list(h3.polyfill(polygon_geojson, h3_resolution))
# busan_h3_cells=busan_h3_cells+(list(h3.polyfill(polygon_geojson2, h3_resolution)))



# #높이 값 추출
# height=[]
# AABB=dem_data.bounds
# data=dem_data.read(1)
# for index in busan_h3_cells:
#     try:
#         coordinates=h3.h3_to_geo(index)
#         coordinate=[coordinates[1],coordinates[0]]
#         coordinates=coordinate
#         coordinates=Get_WGS84(coordinates[0],coordinates[1])
#         index=dem_data.index(coordinates[0],coordinates[1])
#         height.append(data[index[0]][index[1]])
#     except:
#         height.append(0)

# # d=pd.Series(busan_h3_cells,height)
# # d.name="hex_id"
# d={
#     "hex_id":busan_h3_cells,"height":height,
# }




# busan_gdf=pd.DataFrame(d)

# map_1.add_data(data=busan_gdf,name="H3 cells")
# map_1.save_to_html(file_name="h3_.html")